1、不同的文档，使用不同的文档加载器（熟悉）

txt文档: TextLoader

pdf文档: PyPDFLoader

csv文档: CSVLoader

json文档: JSONLoader

html文档: UnstructuredHTMLLoader

md文档: UnstructuredMarkdownLoader

文件目录: DirectoryLoader

2、创建好XxxLoader的实例以后，都需要调用 load()，在内存中返回一个 list[Document]

# 1、加载Txt文档

In [ ]:
from langchain_community.document_loaders import TextLoader

# 指明txt文档的路径
file_path = "./asset/load/01-langchain-utf-8.txt"

# 创建一个TextLoader的实例
text_loader = TextLoader(
    file_path = file_path,
    encoding = "utf-8",
)

# 调用load(), 返回一个list[Document]
docs = text_loader.load()

In [ ]:
# 显示Document对象的元数据
print(docs[0].metadata)

# 显示文档中的内容信息
print(docs[0].page_content)

In [ ]:
from langchain_community.document_loaders import TextLoader

# 指明txt文档的路径
file_path = "./asset/load/01-langchain-gbk.txt"

# 创建一个TextLoader的实例
text_loader = TextLoader(
    file_path = file_path,
    encoding = "gbk", # 此时使用的解码集一定要与当初存储此文件使用的编码集相同, 否则报错!
)

# 调用load(), 返回一个list[Document]
docs = text_loader.load()

print(docs)

# 2、加载pdf文档

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader(
    file_path="./asset/load/02-load.pdf"
)

docs = pdf_loader.load()

print(docs)

从网上调

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader

pdf_loader = PyPDFLoader(
    file_path="https://arxiv.org/pdf/2302.03803"
)

docs = pdf_loader.load()

print(docs)

# 3、加载CSV文档

In [ ]:
from langchain_community.document_loaders import CSVLoader

csv_loader = CSVLoader(
    file_path="./asset/load/03-load.csv"
)

docs = csv_loader.load()

print(len(docs))

for doc in docs:
    print(doc)

# 4、加载JSON文档

举例1: 加载json文件中的所有的数据

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./asset/load/04-load.json",
    jq_schema=".", # 表示加载所有的字段
    text_content=False, # 将加载的json对象转换为json字符串
)

docs = json_loader.load()

print(docs)

举例2:

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./asset/load/04-load.json",
    jq_schema=".messages[].content", # 加载messages[]的所有的content字段
    text_content=False, # 将加载的json对象转换为json字符串
)

docs = json_loader.load()

for doc in docs:
    print(doc.page_content)

举例3: 提取04-response.json文件中嵌套在 data.items[].content 的文本

In [ ]:
from langchain_community.document_loaders import JSONLoader
# 方法一
# json_loader = JSONLoader(
#     file_path="./asset/load/04-response.json",
#     jq_schema=".data.items[].content", # .data.items[].content
# )
# 方法2
# 方式2:
json_loader = JSONLoader(
    file_path="./asset/load/04-response.json",
    jq_schema=".data.items[]", # data.items[].content
    content_key=".content",
    is_content_key_jq_parsable=True, # 用jq解析content_key
)


docs = json_loader.load()

for doc in docs:
    print(doc.page_content)

举例4：提取04-response.json文件中嵌套在 data.items[] 里的 title、content 和 其文本

In [ ]:
# 1.导入相关依赖
from langchain_community.document_loaders import JSONLoader
from pprint import pprint
# 2.定义json文件的路径
file_path = 'asset/load/04-response.json'
# 3.定义JSONLoader对象
# 提取嵌套在 data.items[].content 的文本，并保留其他字段作为元数据
# loader = JSONLoader(
# file_path=file_path,
# # jq_schema=".data.items[] | {id, author, text: (.title + '\n' + .content)}",
# jq_schema='''.data.items[] | {
# id,
# author,
# created_at,
# title, # 保留title字段
# text: (.title + "\n" + .content)
# }''',
# content_key=".text", # 再从条目中提取 content 字段
# is_content_key_jq_parsable=True # 用jq解析content_key
# )
loader = JSONLoader(
    file_path=file_path,
    # jq_schema=".data.items[] | {id, author, text: (.title + '\n' + .content)}",
    jq_schema=".data.items[]",
    content_key='.title + "\\n\\n" + .content',
    is_content_key_jq_parsable=True # 用jq解析content_key
) #
# loader = JSONLoader(
# file_path=file_path,
# # jq_schema=".data.items[] | {id, author, text: (.title + '\n' + .content)}",
# jq_schema='''
# .data.items[] | {
# metadata: {
# id,
# author,
# created_at
# },
# content: (.title + "\n\n" + .content)
# }
# ''', # 构建新结构
# content_key='.title + "\\n\\n" + .content',
# is_content_key_jq_parsable=True # 用jq解析content_key
# )
# 4.加载
data = loader.load()

for doc in data:
    print(doc.page_content)

# 5、加载HTML文档